In [ ]:
import json
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance

In [ ]:
qa = np.genfromtxt("uid_qa.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U128'),])

In [ ]:
#qa[971]

In [ ]:
fe = np.genfromtxt("uid_pre_elim.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U256'),])

In [ ]:
fe.dtype.names = ('uniqueID', 'bn', 'qn', 'pre_que', 'curr_elim')

#fe.dtype.names
#fe[971]

In [ ]:
ftd = np.genfromtxt("face_id_descr.txt", delimiter = ";", skip_header = 1 , usecols = np.arange(0,2), dtype = [('U16'), ('U2056')])

In [ ]:
ftd.dtype.names = ('img_id', 'description')
len(ftd['img_id'])
len(set(ftd['img_id']))

In [ ]:
## first convert strings to list of rows equal to rows in fe
def mk_list(str_vector):
    des_byid = str_vector
    all_x = []
    for i in des_byid:
        j = i.split()
        #if len(j) > 0:
        all_x.append(j)

    return all_x


In [ ]:
all_x = mk_list(fe['pre_que'])
len(all_x)
all_x[972]

In [ ]:
##check blanks
## create blank list
bl =[]
for l in range(len(all_x)):
    if len(all_x[l]) <2:
        bl.append(l)
        
        #print(l)
for l in range(len(qa['que'])):
    if len(qa['que'][l]) <2:
        bl.append(l)
list(set(bl))

In [ ]:
## remove blanks from all 
        
     

In [ ]:
## Now for each row in the list above, find description and add it
all_i_txt = []

for rown in range(len(all_x)):
    #if rown not in bl:
        #print(all_x.index)
    int_i_txt = []
    for img in all_x[rown]:
        if img in ftd['img_id']:
            d = str(ftd['description'][list(ftd['img_id']).index(img)])
            int_i_txt.append(str(d))
    all_i_txt.append(int_i_txt)
        
            
            

In [ ]:
#print check
#print(type(all_i_txt[0]))
print(len(all_i_txt))
print(all_i_txt[971])

In [ ]:
## add question and answer to each data point
for it in all_i_txt:
    it.extend([qa['que'][all_i_txt.index(it)], qa['ans'][all_i_txt.index(it)]])
    
    

In [ ]:
## check addition of above objects
for i in range(len(all_i_txt)):
    print(all_i_txt[i][-2:])

In [ ]:
#for i in set(bl):
#    print(all_i_txt[i][-2:])

In [ ]:
#print(all_i_txt[0])
##basic model
cv = CountVectorizer()

In [ ]:
## all similarities
all_sim = []
for l in range(len(all_i_txt)):
    H = cv.fit_transform(all_i_txt[l])
    sim = cosine_similarity(H)
    all_sim.append(sim)

In [ ]:
#similarities[-2] meaning taking only the array for question
all_sim_q = []
for i in range(len(all_sim)):
    all_sim_q.append(all_sim[i][-2][:-2])
print(len(all_sim_q))
#print(all_sim_q[971])
#print(len(all_sim))

In [ ]:
all_cheat = mk_list(fe['curr_elim'])

In [ ]:
cheat_code = zip(all_sim_q, all_cheat)

In [ ]:
## Sort all the above similarities with argsort which gives indices
## or we should not take top5 and make a threshold above which we will include it in the list of output?
sorted_ind = []
for si, ln in cheat_code:
    y_top = si.argsort()[::-1][:len(ln)]
    sorted_ind.append(y_top)
#sorted_ind[971]


The problem is below. I still have cases where there is only one image id. 
Even though I have made a blank list above. Where I have found out cases where there is no image id before asking the question or one image id before asking the que. Or where que is blank. 

In [ ]:
top_img = zip(all_x, sorted_ind)


In [ ]:
## convert to img_id from index to match with actual image ids selcted by participant
def mk_imgid(arr):
    y_pred = []
    for i,j in arr:
        yp = []
        for s in j:
            yp.append(i[s])
        y_pred.append(yp)
    return y_pred
    
    

In [ ]:
pred_img = mk_imgid(top_img)

In [ ]:
#print(all_cheat[0:3])
#v = mk_imgid(top_img)
#print(w)

In [ ]:
acc_in = zip(all_cheat, pred_img)

In [ ]:
acc_all = []
for a,p in acc_in:
    #print(a,p)
    if len(a) == 0:
        acc_i = 0
    else:
        acc_i = len(set(a).intersection(set(p)))/len(set(a))
    acc_all.append(acc_i)

In [ ]:
#len(pred_img)
print(acc_all)

In [ ]:
np.mean(np.array(acc_all))